# 準備

## Google Colab 用の処理

下記を実行します
- ドライブのマウント
- ノートブックファイルと同じフォルダへの移動 

Googleドライブのマイドライブ を基準に DNN_code/DNN_code_colab_day2 フォルダを置くことを仮定しています。必要に応じて，パスを変更してください．

In [ ]:
# Google Colab での実行かを調べる
import sys
import os
ENV_COLAB = True  if 'google.colab' in sys.modules else False 

# google drive のマウント
if ENV_COLAB:
  from google.colab import drive 
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/DNN_code/DNN_code_colab_day2/notebook')

# モデルの Save/Load 

2_app_1の線形回帰をもとにパラメータのセーブとロードを行う。
save_load/weights-x に保存 ( x は、tf.train.Checkoints により振られるインデックス) 

### モデルの学習とSave

In [ ]:
# %% 
import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(False)

# %%
# データを生成
n = 100
x = np.random.rand(n).astype(np.float32)
d = x *  3 + 2

# ノイズを加える
noise = 0.3
d = d + noise * np.random.randn(n).astype(np.float32) 


# %% 
# モデル
@tf.function
def model_function(x, W, b):
    return W * x + b

# 誤差関数
@tf.function
def loss_fn(y,d):
    return tf.reduce_mean( tf.square(y-d) )

# 最適化の対象の変数を初期化
W = tf.Variable(tf.zeros([1]), name='W')
b = tf.Variable(tf.zeros([1]), name='b')
trainable_variables = [W,b]

# %%
# 最適化関数に SGD を利用
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# トレーニングループの定義
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        y = model_function(inputs, W, b)
        loss = loss_fn(y, labels)
    
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss

# %% トレーニング
iters_num = 100
plot_interval = 10

log_loss = []
for i in range(iters_num):
    loss_val = train_step(x, d)
    log_loss.append(loss_val)
    if (i+1) % plot_interval == 0:
        print('Generation: ' + str(i+1) + '. 誤差 = ' + str(loss_val))


# %% check output
import matplotlib.pyplot as plt 
plt.plot(log_loss)
plt.xlabel("iterations")
plt.ylabel("loss value")

print("W:", W.numpy())
print("b:", b.numpy())

# %% save checkpoint
checkpoint = tf.train.Checkpoint(W=W, b=b)
checkpoint.save('save_load/weights')



モデル・データに関連する gloabal 変数を一度削除

In [ ]:
del W, b, x, d, trainable_variables, optimizer

### モデルの復元

あらためてゼロ初期化した W, b を用意し、モデルの復元前後のモデルの振る舞いを確認する。

In [ ]:
# %%
import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(False)

# %% 
# データを生成
n = 100
x = np.random.rand(n).astype(np.float32)
d = x *  3 + 2

# ノイズを加える
noise = 0.3
d = d + noise * np.random.randn(n).astype(np.float32) 

# %%
# モデル
@tf.function
def model_function(x, W, b):
    return W * x + b

# 誤差関数
@tf.function
def loss_fn(y,d):
    return tf.reduce_mean( tf.square(y-d) )

# ゼロクリアされたモデルパラメータ
W = tf.Variable(tf.zeros([1]), name='W')
b = tf.Variable(tf.zeros([1]), name='b')

# %%
# 関数の出力とパラメータをチェックする関数
def check_model():
    y = model_function(x, W, b)
    loss_val = loss_fn(y, d)

    print('loss_val', loss_val)

    print("W:", W.numpy())
    print("b:", b.numpy())

# %% モデル復元前後で、チェック

print("モデル復元前:")
check_model() 

# restore from checkpoints 
checkpoint = tf.train.Checkpoint(W=W, b=b)
checkpoint.restore('save_load/weights-1')

print("モデル復元後") 
check_model()

